# Datenanalyse von Mess- und Flächennutzungsdaten             <img src="https://www.btelligent.com/fileadmin/_processed_/7/0/csm_data-science-ansatz_2350412d17.jpg" style="max-width:20%">


Zunächst importieren wir wieder die nötigen Pakete, definieren unseren Pfad (bzw. den Dateinnamen wenn im selben Ordner) und laden die CSV-Datei als Pandas-Dataframe.

In [1]:
import pandas as pd
import numpy as np

path = "your_qgis_output.csv"

data = pd.read_csv(path, sep = ",")
print(data.head())

   distance  Concentrat  Conc-min  Conc_rel strassenkl    dtvw_kfz_m  \
0      50.0      6881.5    4494.0     0.985        III  13533.333333   
1     150.0      7169.5    4782.0     1.026        III  14250.000000   
2     250.0      6451.5    4064.0     0.924        III  15000.000000   
3     350.0      6901.5    4514.0     0.988        III  15000.000000   
4     450.0      5906.0    3518.0     0.846        III  15150.000000   

   dtvw_lkw_m    traff_mean  traff_norm  
0       630.0  18573.333333    0.120633  
1       662.5  19550.000000    0.127050  
2       650.0  20200.000000    0.131300  
3       650.0  20200.000000    0.131300  
4       520.0  19310.000000    0.125500  


## Long data format        <img src="https://www.joyofdata.de/blog/wp-content/uploads/2012/11/Clipboard16.png" style="max-width:100%">


Die Daten befinden sich momentan noch im sogenannten long data format. Wir werden sie in diesem weiterverarbeiten und anschließend in das wide data format umwandeln. Zum Unterschieden [hier](https://www.theanalysisfactor.com/wide-and-long-data/) noch eine kurze Erläuterung.

Zunächst benennen wir die Spalten neu, um Missverständnisse zu vermeiden. **Solltet ihr von meiner Reihenfolge und Benennung abgewichen sein, passt die Benennung hier bitte an.**

In [2]:
data.columns = ['trackpoint', 'ufp', 'street_class', 'traffic_norm']
print(data.head())

ValueError: Length mismatch: Expected axis has 9 elements, new values have 4 elements

# Analysen            <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/0/05/Scikit_learn_logo_small.svg/200px-Scikit_learn_logo_small.svg.png" style="max-width:100%">
In diesem Dataframe liegen nun alle Informationen vor, wie wir sie brauchen. Wir können also damit beginnen den Datensatz näher zu analysieren und Hypothesen zu untersuchen. Die folgenden Zeilen zeigen euch, wie ihr in Python mit den Paketen `scikit-learn` und `matplotlib` eine Regression durchführt. Ihr könnt hier aber auch bereits gelerntes aus der letzten Sitzung anwenden und den Datensatz auf weitere Hypothesen untersuchen.
Zunächst schauen wir uns einen linearen Zusammenhang zwischen metrischen Daten an.

**Beachte:** *Es handelt sich hier nicht um einen aufgehübschten Beispieldatensatz sondern um echte Messdaten, wie sie viele von euch auch erhalten werden. Wenn eure Plots also nicht aussehen wie aus dem Lehrbuch und auch die Statistiken eher enttäuschen seid ihr vermutlich auf dem richtigen Weg ;)*

## Lineare Regression

In [3]:
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

Ein Blick auf die Spaltennamen gibt uns einen Überblick, welche Variablen wir miteinander vergleichen können.

In [ ]:
print(data.columns.values)

Der gemessene Ultrafeinstaubwert (`'ufp'`) wird also jeweils unsere abhängige Variable (oder "Predictand", `Y`) sein, das Verkehrsaufkommen oder dei Straßenklasse jeweils die unabhängige Variable (oder "Predictor", `X`). Da wir verschiedene Variablen betrachten wollen, schaffen wir hierfür eine separate Variable (`predictor`) und müssen im Anschluss noch nur diese ändern, um uns weitere Analysen ausgeben zu lassen. Wir beginnen mit dem Verkehrsaufkommen.

Der Datensatz enthält noch ziemlich viele NaN, da wir ja nur für knapp die Hälfte der Messpunkte Verkehrsdaten haben. Für die Regression müssen wir diese Werte entfernen.

Zunächst nutzen wir Boolean Indexing, um alle Zeilen zu markieren, in denen es keine UFP-Daten gibt (also wo `isna() = True` gilt).

In [ ]:
where_nan = data['ufp'].isna()
print(where_nan)

Nun erstellen wir ein Subset der Datentabelle, in der alle Zeilen Verkehrsdaten haben. Die `~` kehrt den Boolean Index dabei um: `NaN = False`. Da wir die vollständigen Daten nicht mehr brauchen überschreiben wir die Originaltabelle mit dem Subset (*von 454 auf 436 Zeilen*).

In [ ]:
data = data[~where_nan]
print(data)

Für die Regressionsanalyse beginnen wir zunächst mit dem Verkehrsaufkommen.

In [ ]:
predictor = 'traffic_norm'

Dafür filtern wir alle Zeilen, in denen es keine Verkehrsdaten gibt (*von 436 auf 211 Zeilen!*).

**Beachte:** *Im Gegensatz zu den Flächenanteilen dürfen wir die Werte nicht 0 setzen! Dies würde bedeuten wir gehen überall, wo es keine Zählungen gab von keinem Verkehr aus, was die Regression massiv verfälschen würde.*

In [ ]:
traffic_subset = data[~data['traffic_norm'].isna()]
print(traffic_subset)

Für die Regression-Funktionen müssen wir die zu untersuchenden Spalten in NumPy-Arrays umwandeln.

In [ ]:
X = traffic_subset.loc[:, predictor].values.reshape(-1, 1)
Y = traffic_subset.loc[:, 'ufp'].values.reshape(-1, 1)

Im nächsten Schritt instanzieren wir die Datenklasse `LinearRegression`.

In [ ]:
linear_regressor = LinearRegression()

Nun trainieren wir unsere lineare Regression (`.fit`) und speichern alle Informationen über unseren Regressor (oder "Predictor") in der Variable `Y_pred`.

In [ ]:
linear_regressor.fit(X, Y)
Y_pred = linear_regressor.predict(X)

Zunächst lassen wir uns das Ergebnis in einem Scatterplot mit der Regressionsgeraden anzeigen. Vereinfacht kann man sagen, die Regressionsgerade beschreibt die lineare Funktion (`Y = a*X + b`), mit der sich die Ultrafeinstaubkonzentration mittels des mittleren Verkehrsaufkommens im Umkreis von 100m um den Messpunkt vorhersagen lässt.

In [ ]:
plt.scatter(X, Y)
plt.plot(X, Y_pred, color='red')
plt.xlabel(str('Mittleres Verkehrsaufkommen' + ' 100m um den Messpunkt'))
plt.ylabel('UFP-Konzentration [#/cm^3]')
plt.show()

Der Plot gibt uns eine Ahnung von der Verteilung der Messwerte und der Richtung und Stärke des Zusammenhangs. Wir erhalten aber keine Informationen über die Güte des linearen Modells. Darum lassen wir uns die `Funktion` des Modells und den Wert `R²` ausgeben. Letzterer gibt uns ein Maß für die Stärke des Zusammenhangs von X und Y.

In [ ]:
print('------ Lineare Regression -----')
print('Funktion: y = %.3f * x + %.3f' % (linear_regressor.coef_[0], linear_regressor.intercept_))
print("R² Score: {:.2f}".format(linear_regressor.score(X, Y)))
print("\n")

Was sehen wir hier? Die Korrelation der Größen ist gering aber vorhanden. Probieren wir es mit einem weiteren Datensatz.

## Lineare Regression mit ordinalen Prädiktoren

Da die Straßenklassen ordinalskalierte Daten sind müssen wir hier mit entsprechenden Funktionen vorgehen.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

predictor = 'street_class'
where_nan = data[predictor].isna()

print(data[where_nan])

In [ ]:
data[predictor].fillna('No_street', inplace=True)

print(data[where_nan])


In [ ]:
X = data.loc[:, predictor].values.reshape(-1, 1)
Y = data.loc[:, 'ufp'].values.reshape(-1, 1)

In [ ]:
enc = OneHotEncoder(sparse=False)
X_trans = enc.fit_transform(X)

linear_regressor = LinearRegression(fit_intercept=False)
linear_regressor.fit(X_trans, Y)

print("Mean squared error: %.2f" % np.mean((linear_regressor.predict(X_trans) - Y) ** 2))

print(linear_regressor.coef_[0])
print(linear_regressor.intercept_)

In [ ]:
print(data.iloc[:, :-1])
print(data.iloc[:, :])


In [ ]:
print('------ Lineare Regression -----')
# print('Funktion: y = %.3f * x + %.3f' % (linear_regressor.coef_[0], linear_regressor.intercept_))
print("R² Score: {:.2f}".format(linear_regressor.score(X, Y)))
print("\n")



In [ ]:
# Kendalls tau und pearson zuerst, dann Regression
# Normalverteilung? --> Bedingungen für lineare Regression checken!?
# UFP auch in Klassen einteilen um Verteilung rauszubekommen


